In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd

In [23]:
# data = pd.read_csv('data/kaggle_data/chessData.csv')
data = pd.read_csv('data/chessdata_500k.csv')

In [24]:
data.head()

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


In [25]:
class ChessDataset(torch.utils.data.Dataset):
    def __init__(self, board, score):
        self.board = board
        self.score = score

    def __len__(self):
        return len(self.score)

    def __getitem__(self, index):
        # print(self.data[index])
        board, score = self.board[index], self.score[index]

        piece_val_map = {'p':-1,'k':-200,'b':-3,'r':-5,'q':-9,'n':-3,'P':1,'K':200,'B':3,'R':5,'Q':9,'N':3}

        input_data = []
        for char in board.split(" ")[0]:
            if char.isdigit():
                for _ in range(int(char)):
                    input_data.append(0)
            else:
                if char == '/':
                    continue
                input_data.append(piece_val_map[char])

        if board.split(" ")[1] == "w":
            input_data.append(1)
        else:
            input_data.append(0)

        if score.startswith("#"):
            score = score[1:]
        
        score= score.replace('\ufeff', '')
            
        data = torch.tensor(input_data, dtype=torch.float32)
        label = torch.tensor(int(score), dtype=torch.float32)

        return data, label

In [26]:
dataset = ChessDataset(data['FEN'], data['Evaluation'])

In [27]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, pin_memory=False)

In [28]:
for data in dataloader:
    print(data[0][0])
    break

tensor([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,   -1.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
        -200.,   -1.,    0.,   -1.,    0.,  200.,    0.,    0.,    0.,    3.,
           0.,    0.,   -1.,    0.,    0.,    0.,   -5.,   -3.,    1.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    1.,    5.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    1.])


In [29]:
class EvalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(65, 1024)
        self.hidden_layer = nn.Linear(1024, 512)
        self.output_layer = nn.Linear(512, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = nn.functional.relu(x)
        x = self.hidden_layer(x)
        x = nn.functional.relu(x)
        x = self.output_layer(x)
        return x

In [30]:
def train(model, dataloader, optimizer, criterion, epochs, device):
    loss_list = []
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(dataloader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = model(data)

            # Loss
            loss = criterion(output, target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Gradient descent
            optimizer.step()
        
        loss_list.append(loss.item())
        running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} loss: {loss.item()} runnning_loss: {running_loss/len(dataloader)}")
    return loss_list

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
model = EvalNetwork()
model.to(device)

EvalNetwork(
  (input_layer): Linear(in_features=65, out_features=1024, bias=True)
  (hidden_layer): Linear(in_features=1024, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=1, bias=True)
)

In [33]:
# create the loss function
criterion = nn.MSELoss()

# create the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# num of epochs
epochs = 100

# train the model
loss_list = train(model, dataloader,  optimizer, criterion,epochs, device)


c:\Users\dahiy_uokx4\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\dahiy_uokx4\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100 loss: 168404.0 runnning_loss: 43.10314819554645
Epoch 2/100 loss: 68533.046875 runnning_loss: 17.54109211031482
Epoch 3/100 loss: 147437.34375 runnning_loss: 37.73671455080625
Epoch 4/100 loss: 285900.96875 runnning_loss: 73.1765980931661
Epoch 5/100 loss: 86262.7265625 runnning_loss: 22.07901882838495
Epoch 6/100 loss: 92546.3671875 runnning_loss: 23.68732203416944
Epoch 7/100 loss: 1234666.625 runnning_loss: 316.0139813155874
Epoch 8/100 loss: 76189.4296875 runnning_loss: 19.500749856027642
Epoch 9/100 loss: 148507.09375 runnning_loss: 38.01051798054773
Epoch 10/100 loss: 128308.78125 runnning_loss: 32.84074257742513
Epoch 11/100 loss: 100572.2265625 runnning_loss: 25.741547622856412
Epoch 12/100 loss: 113901.65625 runnning_loss: 29.153226580496543
Epoch 13/100 loss: 107153.078125 runnning_loss: 27.425922222933195
Epoch 14/100 loss: 743469.25 runnning_loss: 190.29159201433325
Epoch 15/100 loss: 32026.6171875 runnning_loss: 8.19724012989506
Epoch 16/100 loss: 56655.5625 ru

In [34]:
with torch.no_grad():
    y_eval = model.forward(torch.tensor([0.,0.,0.,0.,-5.,   -3., -200.,    0.,    0.,    0.,
          -1.,   -9.,    0.,   -1.,   -1.,    0.,   -1.,    0.,   -3.,    0.,
          -5.,    0.,    0.,   -1.,    0.,   -1.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    1.,    0.,    0.,    3.,   -1.,    0.,    0.,
           1.,    0.,    1.,    1.,    0.,    3.,    0.,    0.,    0.,    0.,
           9.,    0.,    0.,    1.,    1.,    1.,    5.,    0.,    0.,    5.,
           0.,    0.,  200.,    0.,    0.], dtype=torch.float32).to(device))

In [35]:
y_eval

tensor([39.8190], device='cuda:0')

In [36]:
torch.save(model.state_dict(), "model/nn_1024_512_other.pt")

In [24]:
model

EvalNetwork(
  (input_layer): Linear(in_features=65, out_features=1024, bias=True)
  (hidden_layer): Linear(in_features=1024, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
model.output_layer.weight.shape